In [1]:
import pandas as pd
import numpy as np

In [4]:
data =  'crime_data_clean/crime_final_clean10_17.csv'

In [5]:
df = pd.read_csv(data)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006326 entries, 0 to 1006325
Data columns (total 12 columns):
Date           1006326 non-null object
Beat           1006326 non-null object
BlockRange     1006169 non-null object
StreetName     1006300 non-null object
OffenseType    1006326 non-null object
Premise        1006326 non-null object
NumOffenses    1006326 non-null float64
Hour           1006326 non-null float64
day            1006326 non-null int64
weekday        1006326 non-null object
month          1006326 non-null object
year           1006326 non-null int64
dtypes: float64(2), int64(2), object(8)
memory usage: 92.1+ MB


## Select Beats
- create list with select police beats where selected stadiums are located
- filter dataframe with those beats

In [8]:
selected_beats = ['3B10','1A10','10H30','10H50','10H60','10H80','15E40' ]

In [9]:
sel_df = df[df['Beat'].isin(selected_beats)]

In [11]:
sel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77808 entries, 3 to 1006323
Data columns (total 12 columns):
Date           77808 non-null object
Beat           77808 non-null object
BlockRange     77685 non-null object
StreetName     77808 non-null object
OffenseType    77808 non-null object
Premise        77808 non-null object
NumOffenses    77808 non-null float64
Hour           77808 non-null float64
day            77808 non-null int64
weekday        77808 non-null object
month          77808 non-null object
year           77808 non-null int64
dtypes: float64(2), int64(2), object(8)
memory usage: 7.7+ MB


## Inspect BlockRange

In [18]:
len(sel_df[sel_df.BlockRange.isnull()])

123

In [17]:
sel_df[sel_df.BlockRange.isnull()]

,Date,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year
743,2007-07-01,1A10,NaN,LITTLE YORK RD,Theft,No label,1.0,10.0,1,Sunday,Jul,2007
450007,2013-06-11,1A10,NaN,HAYES,Theft,No label,1.0,11.0,11,Tuesday,Jun,2013
456821,2013-06-29,1A10,NaN,RUSK,Theft,No label,1.0,15.0,29,Saturday,Jun,2013
510402,2013-11-23,1A10,NaN,METROPOLITAN,Theft,No label,1.0,8.0,23,Saturday,Nov,2013
523954,2014-01-01,1A10,NaN,TIDWELL RD,Theft,No label,1.0,13.0,1,Wednesday,Jan,2014
561982,2014-04-21,1A10,NaN,HUNTINGTON VENTURE DR,Theft,No label,1.0,14.0,21,Monday,Apr,2014
563663,2014-04-26,1A10,NaN,JENSEN DR / SAUNDERS,Theft,No label,1.0,11.0,26,Saturday,Apr,2014
567756,2014-05-08,1A10,NaN,LOCKWOOD,Theft,No label,1.0,12.0,8,Thursday,May,2014
570691,2014-05-16,1A10,NaN,HOLLY HALL,Theft,No label,1.0,8.0,16,Friday,May,2014
570706,2014-05-16,1A10,NaN,W SAM HOUSTON PKWY OB TO SOUTH,Theft,No label,1.0,12.0,16,Friday,May,2014


----

# Save Final Clean Data

In [19]:
sel_df.to_csv('crime_data_clean/beats_filter_10_17.csv')